In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from lsst.rubintv.production.plotting.nightReportPlotBase import StarTrackerPlot
%matplotlib inline

In [ ]:
dayObs = 20230221
mdFilename = os.path.join('/project/rubintv/starTracker/sidecar_metadata', f'dayObs_{dayObs}.json')
metadata = pd.read_json(mdFilename).T
metadata = metadata.sort_index()

In [ ]:
# any classes added to __all__ will automatically be added to the night
# report channel, with each being replotted for each image taken.
__all__ = ['RaDecAltAzOverTime',
           'DeltasPlot',
           'SourcesAndScatters',
           'AltAzCoverageTopDown',
           'AltAzCoverage',
           ]

COLORS = 'bgrcmyk'  # these get use in order to automatically give a series of colors for data series

In [ ]:
class AltAzCoverageTopDown(StarTrackerPlot):
    _PlotName = 'Alt-Az-top-down'
    _PlotGroup = 'Coverage'

    def __init__(self,
                 dayObs,
                 locationConfig=None,
                 uploader=None):
        super().__init__(dayObs=dayObs,
                         plotName=self._PlotName,
                         plotGroup=self._PlotGroup,
                         locationConfig=locationConfig,
                         uploader=uploader)

    def plot(self, metadata):
        """Create a sample plot using data from the StarTracker page tables.

        Parameters
        ----------
        metadata : `pandas.DataFrame`
            The data from all three StarTracker page tables, as a dataframe.

        Returns
        -------
        success : `bool`
            Did the plotting succeed, and thus upload should be performed?
        """
        # TODO: get a figure you can reuse to avoid matplotlib memory leak
        _ = plt.figure(figsize=(10, 10))
        ax = plt.subplot(111, polar=True)

        alts = metadata['Alt']
        azes = metadata['Az']
        ax.plot([az*np.pi/180 for az in azes], alts, 'or', label='Pointing')
        hasWideSolve = metadata.dropna(subset=['Calculated Dec wide'])
        wideAlts = hasWideSolve['Alt']
        wideAzes = hasWideSolve['Az']
        ax.scatter([az*np.pi/180 for az in wideAzes], wideAlts, marker='o', s=200, facecolors='none', edgecolors='b', label='Wide Solve')
        hasNarrowSolve = metadata.dropna(subset=['Calculated Dec'])
        narrowAlts = hasNarrowSolve['Alt']
        narrowAzes = hasNarrowSolve['Az']
        ax.scatter([az*np.pi/180 for az in narrowAzes], narrowAlts, marker='o', s=400, facecolors='none', edgecolors='g', label='Narrow Solve')

        ax.set_title("Axial coverage - azimuth (theta) vs altitude(r)"
                     "\n 'Top down' view with zenith at center", va='bottom')
        ax.set_theta_zero_location("N")
        ax.set_theta_direction(-1)
        ax.set_rlim(0, 90)
        ax.legend()

        ax.invert_yaxis()  # puts 90 (the zenith) at the center
        return True

In [ ]:
plot = AltAzCoverageTopDown(dayObs)
plot.plot(metadata)
plt.savefig("/home/craiglagegit/DATA/StarTracker_Coverage_Plot_22Feb23.png")

In [ ]:
metadata.columns

In [ ]:
metadata['Calculated Dec wide'].values

In [ ]:
md2 = metadata.dropna(subset=['Calculated Dec wide'])

In [ ]:
md2